In [ ]:
pip install apache-airflow

     |████████████████████████████████| 5.3 MB 26.9 MB/s 
     |████████████████████████████████| 55 kB 3.6 MB/s 
     |████████████████████████████████| 3.5 MB 59.1 MB/s 
     |████████████████████████████████| 133 kB 23.1 MB/s 
     |████████████████████████████████| 155 kB 58.1 MB/s 
     |████████████████████████████████| 111 kB 74.5 MB/s 
     |████████████████████████████████| 548 kB 51.5 MB/s 
     |████████████████████████████████| 56 kB 4.2 MB/s 
     |████████████████████████████████| 208 kB 66.4 MB/s 
     |████████████████████████████████| 211 kB 73.7 MB/s 
     |████████████████████████████████| 6.2 MB 58.3 MB/s 
     |████████████████████████████████| 49 kB 5.9 MB/s 
     |████████████████████████████████| 77 kB 6.1 MB/s 
     |████████████████████████████████| 636 kB 47.2 MB/s 
     |████████████████████████████████| 79 kB 7.8 MB/s 
     |████████████████████████████████| 1.9 MB 48.6 MB/s 
     |████████████████████████████████| 75 kB 4.3 MB/s 
     |████████████████████

In [ ]:
!airflow initdb

usage: airflow [-h] GROUP_OR_COMMAND ...

positional arguments:
  GROUP_OR_COMMAND

    Groups:
      celery         Celery components
      config         View configuration
      connections    Manage connections
      dags           Manage DAGs
      db             Database operations
      jobs           Manage jobs
      kubernetes     Tools to help run the KubernetesExecutor
      pools          Manage pools
      providers      Display providers
      roles          Manage roles
      tasks          Manage tasks
      users          Manage users
      variables      Manage variables

    Commands:
      cheat-sheet    Display cheat sheet
      info           Show information about current Airflow and environment
      kerberos       Start a kerberos ticket renewer
      plugins        Dump information about loaded plugins
      rotate-fernet-key
                     Rotate encrypted connection credentials and variables
      scheduler      Start a scheduler instance
      sync-p

In [ ]:
!airflow webserver

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144 UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
[2021-09-30 10:27:50,226] {cli_action_loggers.py:105} WARNING - Failed to log action with (sqlite3.OperationalError) no such table: log
[SQL: INSERT INTO log (dttm, dag_id, task_id, event, execution_date, owner, extra) VALUES (?, ?, ?, ?, ?, ?, ?)]
[parameters: ('2021-09-30 10:27:50.222504', None, None, 'cli_webserver', None, 'root', '{"host_name": "7b0dff3358fe", "full_command": "[\'/usr/local/bin/airflow\', \'webserver\']"}')]
(Background on this error at: http://sqlalche.me/e/13/e3q8)
  ____________       _____________
 ____    |__( )_________  __/__  /________      __
____  /| |_  /__  ___/_  /_ __  /_  __ \_ | /| / /
___  ___ |  / _  /   _  __/ _  / / /_/ /_ |/ |/ /
 _/_

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip

--2021-09-30 00:56:36--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.161.241.46, 18.205.222.128, 52.202.168.65, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.161.241.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  3.37MB/s    in 6.9s    

2021-09-30 00:56:44 (1.92 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]



In [ ]:
!unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [ ]:
import datetime as dt

from airflow import DAG
from airflow.operators.bash_operator import BashOperator
from airflow.operators.python_operator import PythonOperator

def greet():
  print('writing in file')
  with open('greet.txt', 'a+',encoding='utf8') as f:
    now = dt.datetime.now()
    t = now.strftime("%Y-%m-%d %H:%M")
    f.write(str(t) + '\n')
    return 'Greeted'

def respond():
  return 'Greet Responded Again'

default_args = {
    'owner' : 'airflow',
    'start_date': dt.datetime(2021,9,23,10,00,00),
    'concurrency':1,
    'retries':0
}

with DAG ('my-simple-dag',
          catchup = False,
          default_args = default_args,
          schedule_interval = '*/10 * * * *',  
  ) as dag:

  opr_hello = BashOperator(task_id = 'say_Hi',
                           bash_command = 'echo "Hi!"')
  opr_greet = PythonOperator(task_id = 'greet',
                             python_callable = greet)
  opr_sleep = BashOperator(task_id = 'sleep_me',
                           bash_command = 'sleep 5')
  opr_respond = PythonOperator(task_id = 'respond',
                               python_callable = respond)
  

In [ ]:
opr_respond

<Task(PythonOperator): respond>

In [ ]:
greet()

writing in file


'Greeted'

In [ ]:
# DAG 02

import re
from datetime import datetime, timedelta
import requests 

from airflow import DAG
from airflow.operators.python_operator import BranchPythonOperator
from airflow.operators.dummy_operator import DummyOperator
from airflow.operators.email_operator import EmailOperator

def if_value_higher_3000():
  res = requests.get("https://www.nasdaq.com/")
  SnP500_value = extractor.findall(res.text)[0]
  # As we are going to test email operator,
  # we set it to True
  if True:
    return 'send_email'
  else:
    return 'do_nothing'

default_args = {
    'owner' : 'airflow'
}

dag = DAG ('example_email_operator',
           catchup = False,
           start_date = datetime(2019,1,1),
           schedule_interval = '@daily',
           default_args=default_args)

PATTERN = 'storeIndexInfo\("S&P 500","(.+?)"'
extractor = re.compile(PATTERN)


t_get_SnP500 = BranchPythonOperator(
    
    task_id = 'get_SnP500_value',
    python_callable = if_value_higher_3000,
    dag=dag,
)

t_send_email = EmailOperator(task_id = 'send_email',
                           subject = "Today's S&P 500 value",
                           to = "gilbertlaurito16@gmail.com",
                           
                           html_content = "Hey, it is higher than 3000",
                           dag=dag)

t_do_nothing = DummyOperator(
    task_id = 'do_nothing',
    dag=dag,
)

In [ ]:
t_get_SnP500 >> (t_send_email, t_do_nothing)

(<Task(EmailOperator): send_email>, <Task(DummyOperator): do_nothing>)

In [ ]:
# EX3 

from airflow import DAG
from airflow.operators.bash_operator import BashOperator
from datetime import datetime, timedelta

default_args = {
    'owner': 'muller',
    'depends_on_past': False,
    'start_date': datetime(2021,8,1),
    'email': ['airflow@example.com'],
    'email_on_failure': False,
    'email_on_retry': False,
    'retries': 1,
    'retry_delay': timedelta(minutes=5),
}

dag = DAG('daily_report', default_args = default_args,
          schedule_interval=timedelta(days=1))

t1_a = BashOperator (
    task_id = 'get_db_A',
    bash_command = 'sh script_A.sh',
    dag=dag
)

t1_b = BashOperator (
    task_id = 'get_db_B',
    bash_command = 'sh script_B.sh',
    dag=dag
)

t1_c = BashOperator (
    task_id = 'get_db_C',
    bash_command = 'sh script_C.sh',
    dag=dag)


t3_b = BashOperator (
    task_id = 'get_db_3B',
    bash_command = 'sh script_3B.sh',
    dag=dag
)

t3_c = BashOperator (
    task_id = 'get_db_3C',
    bash_command = 'sh script_3C.sh',
    dag=dag
)

t2 = BashOperator (
    task_id = 'join_data',
    bash_command = 'sh join_data.sh',
    dag=dag
)

t3 = BashOperator (
    task_id = 'analyze',
    bash_command = 'sh analyze.sh',
    dag=dag
)

In [ ]:
(t1_a,t1_b,t1_c)>> t2>> t3

<Task(BashOperator): analyze>

In [ ]:
# EX4
#Azure

!pip install apache-airflow-providers-microsoft-azure

     |████████████████████████████████| 86 kB 5.5 MB/s 
     |████████████████████████████████| 2.3 MB 34.7 MB/s 
     |████████████████████████████████| 106 kB 51.7 MB/s 
     |████████████████████████████████| 228 kB 56.6 MB/s 
     |████████████████████████████████| 61 kB 321 kB/s 
     |████████████████████████████████| 477 kB 59.0 MB/s 
     |████████████████████████████████| 109 kB 62.3 MB/s 
     |████████████████████████████████| 47 kB 5.1 MB/s 
     |████████████████████████████████| 88 kB 7.2 MB/s 
     |████████████████████████████████| 96 kB 5.8 MB/s 
     |████████████████████████████████| 356 kB 76.1 MB/s 
     |████████████████████████████████| 85 kB 3.7 MB/s 
     |████████████████████████████████| 40 kB 5.8 MB/s 
     |████████████████████████████████| 55 kB 3.7 MB/s 
     |████████████████████████████████| 75 kB 4.0 MB/s 
     |████████████████████████████████| 166 kB 52.6 MB/s 
     |████████████████████████████████| 289 kB 60.2 MB/s 
     |██████████████████████████

In [ ]:
from airflow import DAG
from datetime import datetime, timedelta
from airflow.operators.email_operator import EmailOperator
from airflow.operators.python_operator import PythonOperator
from airflow.providers.microsoft.azure.hooks.azure_data_factory import AzureDataFactoryHook
from airflow.providers.microsoft.azure.hooks.wasb import WasbHook

#Get yesterday's date in the correct format

yesterday_date = '{{ yesterday_ds_nodash }}'

#Define Greate Expectations file paths
data_dir = '/usr/local/airflow/include/data/'
data_file_path = '/usr/local/airflow/include/data/'
ge_root_dir = '/usr/local/airflow/include/great_expectations/'


def run_adf_pipeline(pipeline_name,data):
  params= {}
  params["date"] = date

  hook = AzureDataFactoryHook('connection_string')
  hook.run_pipeline(pipeline_name,parameters=params)

def get_azure_blob_files(blobname,output_filename):
  
  azure = WasbHook(wasb_conn_id='azure_blob')
  azure.get_file(output_filename,container_name='data',blob_name=blobname)

default_args = {
    'owner': 'airflow',
    'depends_on_past': False,
    'email_on_failure': False,
    'email_on_retry': False,
    'retries': 0,
    'retry_delay': timedelta(minutes=5)
}


with DAG('adf_great_expectations',
         start_date = datetime(2021,1,1),
         max_active_runs=1,
         schedule_interval='@daily',
         default_args=default_args,
         catchup=False) as dag:


      #    run_pipeline = PythonOperator(
      #       task_id ='run_pipeline',
      #       python_collable=run_adf_pipeline,
      #       op_kwargs={'pipeline_name': 'pipeline1','date': yesterday_date}
      #   )

          run_pipeline = PythonOperator(
              task_id = 'run_pipeline',
              python_callable=run_adf_pipeline,
              op_kwargs={'pipeline_name': 'pipeline1','date': yesterday_date}
          )
          
  #       download_data = PythonOperator(
  #           task_id = 'download_data',
  #           python_collable=get_azure_blob_files,
  #           op_kwargs={'blobname': 'or/' + yesterday_date + '.csv', 'output_filename': data_file_path+'or_'+yesterday_date+'.csv'} 
  #                    
  #       )
          download_data = PythonOperator(
                task_id = 'download_data',
                python_callable=get_azure_blob_files,
                op_kwargs={'blobname': 'or/'+ yesterday_date +'.csv', 'output_filename': data_file_path+'or_'+yesterday_date+'.csv'}
                 
          )

  #       send_email = EmailOperator(
  #           task_id = 'send_email',
  #           to='noreply@astronomer.io',
  #           subject ='S3 DAG',
  #           html_content='<p>The great expectations checks passed successfully. </p>'
                      
  #       )

          send_email = EmailOperator (
              task_id='email1',
              to='noreply@astronomer.io',
              subject='S3 DAG',
              html_content ="<p>The great expectations checks passed successfully. </p>"
          )

run_pipeline>> download_data>>send_email

<Task(EmailOperator): email1>

In [ ]:
# API

from airflow import DAG
from airflow.operators.dummy_operator import DummyOperator
from airflow.operators.python_operator import PythonOperator
from datetime import datetime, timedelta
import time
import requests

def api_function(*kwargs):

  url = 'https://covidtracking.com/api/v1/states'
  filename = '{0}/{1}.csv'.format('wa','2020-03-31')
  res = requests.get(url+filename)

with DAG('pool_dag',
         start_date = datetime(2021,8,1),
         schedule_interval = timedelta(minutes=30),
         catchup = False,
         default_args={
             'retries':1,
             'retry_delay': timedelta(minutes=5)         
        }        
        ) as dag:

    task_w = DummyOperator(
            task_id ='start'
        )


    task_x = PythonOperator(
            task_id ='task_y',
            python_callable=api_function,
            pool='api_pool'
            )
task_w >> task_x

<Task(PythonOperator): task_y>

In [ ]:
# sla DAG

from airflow import DAG
from airflow.operators.dummy_operator import DummyOperator
from airflow.operators.python_operator import PythonOperator
from datetime import datetime, timedelta
import time

def my_custom_functions(ts,*kwargs):
  print("task is sleeping")
  time.sleep(40)

default_args = {
    'owner': 'airflow',
    'depends_on_past': False,
    'email_on_failure': True,
    'email': 'mail@abc.com',
    'email_on_retry': False,
    'sla': timedelta(seconds=30)
}

with DAG('sla-dag',
         start_date=datetime(2021,1,1),
         max_active_runs=1,
         schedule_interval=timedelta(minutes=2),
         default_args=default_args,
         catchup=False
         )as dag:

# 2 DummyOperator / 1 PythonOperator

    t0 = DummyOperator(
        task_id ='start'
    )

    t1 = DummyOperator(
        task_id ='end'
    )

    sla_task = PythonOperator(
        task_id = 'sla_task',
        python_callable = my_custom_functions
    )

t0 >> sla_task >> t1

<Task(DummyOperator): end>